# loading model 
this part needs to output:
1. tokenizer
2. model 
3. max_len

In [1]:
#tramsformers imports
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.23.0")

2023-06-14 15:52:18.188582: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 15:52:18.190647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-14 15:52:18.190652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#model parameters
DEVICE='cpu'
MAX_LEN=4096

In [3]:
#imports
import json
import os
from os.path import join
import pandas as pd
from tqdm.notebook import tqdm

import pygments
from pygments.lexers import get_lexer_by_name

import torch.nn.functional as F 
import torch 


In [4]:
#genral parameters
data_file='cpp000000000302.json'
LANG='cpp'
SAVE_DIR='test_save'

In [5]:
#geting the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-160m")
model = AutoModelForCausalLM.from_pretrained("NinedayWang/PolyCoder-160m").to(DEVICE)

model.config.max_position_embeddings=MAX_LEN
#making_new_bias
b = torch.ones(MAX_LEN, MAX_LEN,dtype=torch.uint8)
b = torch.tril(b)
b = b.unsqueeze(0).unsqueeze(0).to(DEVICE)


for l in model.base_model.layers:
    l.attention.bias=b.clone()

In [6]:
import json

def process_data_file(data_file):
    """
    Read a file containing JSON objects, parse them, and handle any errors encountered.

    Args:
        data_file (str): The path to the data file.

    Returns:
        tuple: A tuple containing the processed data, a list of errors encountered,
               and a list of faulty lines.

    """
    data = []  # List to store the parsed JSON objects
    errors = []  # List to store the encountered errors
    faultys = []  # List to store the faulty lines

    with open(data_file, 'rb') as f:
        for i, line in enumerate(f):
            try:
                data.append(json.loads(line))  # Parse the JSON object and add it to 'data'
            except Exception as e:
                print(f'Errored at line {i}: {e}')
                errors.append(e)
                faultys.append(line)

    #print(f'data: {len(data)} errors: {len(errors)}')
    return data, errors, faultys

# Example usage:

repos, errors, faulty_lines = process_data_file(data_file)
print(f'data: {len(repos)} errors: {len(errors)}')

data: 10108 errors: 0


In [8]:
valid_repos=[d for d in repos if 'content' in d.keys() and 'repo_name' in d.keys()]
#codes=[d['content'] for d in valid_repos]
print(len(valid_repos))

10104


In [9]:
lexer = get_lexer_by_name(LANG)
LEX_VOCAB=sum(len(v) for v in lexer.tokens.values())
LEX_VOCAB

46

# main loop

In [10]:
class CodeTooLongError(ValueError):
    def __init__(self, message="Code too long for the model"):
        self.message = message
        super().__init__(self.message)
#raise CodeTooLongError

In [11]:
def compute_repo_metrics(repo):
    c = repo['content']
    num_tok = len(list(pygments.lex(c, lexer)))
    input_ids = tokenizer.encode(c, return_tensors='pt').to(DEVICE)
    if input_ids.shape[-1] > LEX_VOCAB:
        raise CodeTooLongError
    out = model(input_ids).logits
    l = F.cross_entropy(out, F.one_hot(input_ids, out.shape[-1]).to(float), reduction='sum')
    return {'repo_name': repo['repo_name'], 'cross_entropy': l.detach().cpu().item(), 'lexical_length': num_tok}


In [12]:
ans=[]
skiped=[]
for r in tqdm(valid_repos ):
    try: 
        ans.append(compute_repo_metrics(r))
    except Exception as e:
        skiped.append({'repo_name':r['repo_name'],'error':e})

  0%|          | 0/10104 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
def get_results(ans):
    #need to do numerical stabilety better
    all_tokens=sum(x['lexical_length'] for x in ans)
    neg_log=torch.sum(torch.Tensor(tuple(x['cross_entropy'] for x in ans)))
    preplexity=torch.exp(neg_log/(all_tokens*LEX_VOCAB))
    return {'preplexity':preplexity.detach().cpu().item(),'lex tokens':all_tokens}

results=get_results(ans)
results

{'preplexity': 1.3579456806182861, 'lex tokens': 84}

# record keeping

In [17]:
good_df=pd.DataFrame(ans)
bad_df=pd.DataFrame(skiped)
ans_df=pd.DataFrame([results])

if SAVE_DIR:
    os.makedirs(SAVE_DIR)
    good_df.to_csv(join(SAVE_DIR,'calc.csv'),index=False)
    bad_df.to_csv(join(SAVE_DIR,'error.csv'),index=False)
    ans_df.to_csv(join(SAVE_DIR,'result.csv'),index=False)